In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm

from docplex.mp.model import Model
from docplex.mp.environment import Environment

In [2]:
def cost_function(prediction):
    days = list(range(N_DAYS,0,-1))

    penalty_all = 0
    pref_cost = []

    # We'll use this to count the number of people scheduled each day
    daily_occupancy = {k:0 for k in days}
    
    # Looping over each family; d is the day for each family f
    for f, d in enumerate(prediction):

        # Using our lookup dictionaries to make simpler variable names
        n = family.n_people[f]
        choice_0 = family['choice_0'][f]
        choice_1 = family['choice_1'][f]
        choice_2 = family['choice_2'][f]
        choice_3 = family['choice_3'][f]
        choice_4 = family['choice_4'][f]
        choice_5 = family['choice_5'][f]
        choice_6 = family['choice_6'][f]
        choice_7 = family['choice_7'][f]
        choice_8 = family['choice_8'][f]
        choice_9 = family['choice_9'][f]

        # add the family member count to the daily occupancy
        daily_occupancy[d] += n

        # Calculate the penalty for not getting top preference
        penalty = 0
        if d == choice_0:
            penalty += 0
        elif d == choice_1:
            penalty += 50
        elif d == choice_2:
            penalty += 50 + 9 * n
        elif d == choice_3:
            penalty += 100 + 9 * n
        elif d == choice_4:
            penalty += 200 + 9 * n
        elif d == choice_5:
            penalty += 200 + 18 * n
        elif d == choice_6:
            penalty += 300 + 18 * n
        elif d == choice_7:
            penalty += 300 + 36 * n
        elif d == choice_8:
            penalty += 400 + 36 * n
        elif d == choice_9:
            penalty += 500 + 36 * n + 199 * n
        else:
            penalty += 500 + 36 * n + 398 * n
        penalty_all += penalty
        pref_cost.append(penalty)

    penalty = penalty_all
    # for each date, check total occupancy
    #  (using soft constraints instead of hard constraints)
    for _, v in daily_occupancy.items():
        if (v > MAX_OCCUPANCY) or (v < MIN_OCCUPANCY):
            penalty += 100000000

    # Calculate the accounting cost
    # The first day (day 100) is treated special
    accounting_cost = (daily_occupancy[days[0]]-125.0) / 400.0 * daily_occupancy[days[0]]**(0.5)
    # using the max function because the soft constraints might allow occupancy to dip below 125
    accounting_cost = max(0, accounting_cost)
    
    # Loop over the rest of the days, keeping track of previous count
    yesterday_count = daily_occupancy[days[0]]
    for day in days[1:]:
        today_count = daily_occupancy[day]
        diff = abs(today_count - yesterday_count)
        accounting_cost += max(0, (daily_occupancy[day]-125.0) / 400.0 * daily_occupancy[day]**(0.5 + diff / 50.0))
        yesterday_count = today_count
    
    penalty += accounting_cost

    return penalty, daily_occupancy, accounting_cost, pref_cost

In [3]:
family = pd.read_csv('../input/family_data.csv')
family.head()

,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
0,0,52,38,12,82,33,75,64,76,10,28,4
1,1,26,4,82,5,11,47,38,6,66,61,4
2,2,100,54,25,12,27,82,10,89,80,33,3
3,3,2,95,1,96,32,6,40,31,9,59,2
4,4,53,1,47,93,26,3,46,16,42,39,4


In [4]:
sample = pd.read_csv('../input/sample_submission.csv')

In [5]:
num_family = family.shape[0]
num_days = 100
families = range(num_family)
days = range(1, 101)
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125
attendances = range(MIN_OCCUPANCY, MAX_OCCUPANCY+1)
#MAX_ATTENDANCE_DELTA = 29
MAX_DAY_COST = 68888
#MAX_CHOICE = 10
N_DAYS = 100

In [6]:
accounting_cost = np.zeros((301, 301))
for i in attendances:
    for j in attendances:
        accounting_cost[i, j] = ((i - 125) / 400) * np.power(i, 0.5 + np.abs(i - j) / 50)

In [7]:
last_day_cost = np.zeros((301,))
for i in attendances:
    last_day_cost[i] = ((i - 125) / 400) * np.power(i, 0.5)

In [8]:
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python version 3.7.3, located at: /home/jfpuget/anaconda3/bin/python
* docplex is present, version is (2, 11, 176)
* CPLEX library is present, version is 12.10.0.0, located at: /home/jfpuget/anaconda3/lib/python3.7/site-packages
* pandas is present, version is 0.24.2


In [9]:
mdl = Model("santa")

In [10]:
assign_family_to_day_vars = mdl.binary_var_matrix(families, days, lambda c: '%d_to_%d' % c)

In [11]:
mdl.add_constraints(
    mdl.sum(assign_family_to_day_vars[f, d] * family.n_people[f] for f in families) <= MAX_OCCUPANCY
    for d in days)
mdl.print_information()

Model: santa
 - number of variables: 500000
   - binary=500000, integer=0, continuous=0
 - number of constraints: 100
   - linear=100
 - parameters: defaults
 - problem type is: MILP


In [12]:
mdl.add_constraints(
    mdl.sum(assign_family_to_day_vars[f, d] * family.n_people[f] for f in families) >= MIN_OCCUPANCY
    for d in days)
mdl.print_information()

Model: santa
 - number of variables: 500000
   - binary=500000, integer=0, continuous=0
 - number of constraints: 200
   - linear=200
 - parameters: defaults
 - problem type is: MILP


In [13]:
mdl.add_constraints(
    mdl.sum(assign_family_to_day_vars[f, d] for d in days) == 1
    for f in families)
mdl.print_information()

Model: santa
 - number of variables: 500000
   - binary=500000, integer=0, continuous=0
 - number of constraints: 5200
   - linear=5200
 - parameters: defaults
 - problem type is: MILP


In [14]:
preference_cost = np.load('../input/preference_cost.npy')

In [15]:
n_people = sorted(family.n_people.unique())
n_people

[2, 3, 4, 5, 6, 7, 8]

In [16]:
pred_family = {}

for n in n_people:
    pred_family[n] = -1

In [17]:
choices = np.ones((num_family, num_days+1)).astype('int')
for f in families:
    for i,c in enumerate(['choice_%d' % ci for ci in range(10)]):
        d = family.at[f, c]
        choices[f, d] = 0

In [18]:
# symmetry breaking
for f in tqdm(families):
    n = family.at[f, 'n_people']
    f_prev = pred_family[n]
    pred_family[n] = f
    if f_prev < 0:
        continue
    coef = choices[f] * choices[f_prev]
    coef = coef * np.arange(choices.shape[1])
    mdl.add(1 +
            choices.shape[1] * mdl.sum((1 - choices[f, d]) * assign_family_to_day_vars[f, d] for d in days) +
            choices.shape[1] * mdl.sum((1 - choices[f, d]) * assign_family_to_day_vars[f_prev, d] for d in days) +
            mdl.sum(coef[d] * assign_family_to_day_vars[f, d] for d in days) >=
            mdl.sum(coef[d] * assign_family_to_day_vars[f_prev, d] for d in days)
           )

100%|██████████| 5000/5000 [00:14<00:00, 340.11it/s]


In [19]:
mdl.print_information()

Model: santa
 - number of variables: 500000
   - binary=500000, integer=0, continuous=0
 - number of constraints: 10193
   - linear=10193
 - parameters: defaults
 - problem type is: MILP


In [20]:
day_attendance_bit = mdl.binary_var_matrix(days, attendances, lambda c: 'day_%d_had_%d' % c)
day_attendance_prev = mdl.binary_var_cube(days[:-1], attendances, attendances, #0, 1, 
                                              lambda c: 'day_%d_had_%d_prev_%d' % c)

In [21]:
mdl.add_constraints(
    mdl.sum(day_attendance_bit[d, a] for a in attendances) == 1
    for d in days)
mdl.print_information()

Model: santa
 - number of variables: 3584224
   - binary=3584224, integer=0, continuous=0
 - number of constraints: 10293
   - linear=10293
 - parameters: defaults
 - problem type is: MILP


In [22]:
mdl.add_constraints(
    mdl.sum(assign_family_to_day_vars[f, d] * family.n_people[f] for f in families) == 
    mdl.sum(a * day_attendance_bit[d, a] for a in attendances)
    for d in days)
mdl.print_information()

Model: santa
 - number of variables: 3584224
   - binary=3584224, integer=0, continuous=0
 - number of constraints: 10393
   - linear=10393
 - parameters: defaults
 - problem type is: MILP


In [23]:
mdl.add_constraints(
    day_attendance_bit[d, a_d] == 
    mdl.sum(day_attendance_prev[d, a_d, a_d1] for a_d1 in attendances)
    for a_d in attendances 
    for d in days[:-1])
mdl.print_information()

Model: santa
 - number of variables: 3584224
   - binary=3584224, integer=0, continuous=0
 - number of constraints: 27817
   - linear=27817
 - parameters: defaults
 - problem type is: MILP


In [24]:
mdl.add_constraints(
    day_attendance_bit[d+1, a_d1] == 
    mdl.sum(day_attendance_prev[d, a_d, a_d1] for a_d in attendances)
    for a_d1 in attendances 
    for d in days[:-1])
mdl.print_information()

Model: santa
 - number of variables: 3584224
   - binary=3584224, integer=0, continuous=0
 - number of constraints: 45241
   - linear=45241
 - parameters: defaults
 - problem type is: MILP


In [25]:
accounting_max = mdl.add_constraints(day_attendance_prev[d, a_d, a_d1] == 0 
                             for d in tqdm(days[:-1])
                             for a_d in attendances
                             for a_d1 in attendances
                             if accounting_cost[a_d, a_d1] > MAX_DAY_COST
                    )                    

mdl.print_information()

100%|██████████| 99/99 [00:06<00:00, 21.88it/s]


Model: santa
 - number of variables: 3584224
   - binary=3584224, integer=0, continuous=0
 - number of constraints: 648547
   - linear=648547
 - parameters: defaults
 - problem type is: MILP


In [26]:
last_day_max = mdl.add_constraints(day_attendance_bit[d, a_d] == 0 
                                   for a_d in attendances
                                   for d in days
                                   if last_day_cost[a_d] > MAX_DAY_COST
                    )                    

mdl.print_information()

Model: santa
 - number of variables: 3584224
   - binary=3584224, integer=0, continuous=0
 - number of constraints: 648547
   - linear=648547
 - parameters: defaults
 - problem type is: MILP


In [27]:
if 1:
    accounting_penalty = mdl.sum(accounting_cost[a_d, a_d1] * day_attendance_prev[d, a_d, a_d1]
                             for d in tqdm(days[:-1])
                             for a_d in attendances
                             for a_d1 in attendances
                             if accounting_cost[a_d, a_d1] <= MAX_DAY_COST
                            )

    last_day_penalty = mdl.sum(last_day_cost[a_d] * day_attendance_bit[d, a_d]
                               for d in days[-1:]
                               for a_d in attendances
                               if last_day_cost[a_d] <= MAX_DAY_COST
                            )
    accounting_penalty = accounting_penalty +  last_day_penalty      

else:
    day_cost = mdl.continuous_var_dict(days, name='day_cost', lb=0, #ub=MAX_DAY_COST
                                  )

    mdl.add_constraints(day_cost[d] >= mdl.sum(accounting_cost[a_d, a_d1] * day_attendance_prev[d, a_d, a_d1]
                                         for a_d in attendances
                                         for a_d1 in attendances
                                         if accounting_cost[a_d, a_d1] <= MAX_DAY_COST
                                          )
                             for d in tqdm(days[:-1])
                            )

                    
    for dd in days[-1:]:
        mdl.add(day_cost[dd] >= mdl.sum(last_day_cost[a_d] * day_attendance_bit[d, a_d]
                               for d in days[-1:]
                                 for a_d in attendances
                                 if last_day_cost[a_d] <= MAX_DAY_COST)
                            )

    accounting_penalty = mdl.sum(day_cost[d] for d in days)

100%|██████████| 99/99 [00:10<00:00,  7.62it/s]


In [28]:
mdl.parameters.timelimit = 1000  # nurse should not take more than that !


In [29]:
sub = pd.read_csv('../submissions/submission_68888.04343194816.csv')

In [30]:
# checking sub

sub_cst = mdl.add_constraints(assign_family_to_day_vars[f, a] == 1 for f,a in zip(sub.family_id, sub.assigned_day))

In [31]:
preference_penalty = mdl.sum(assign_family_to_day_vars[f, d] * preference_cost[f, d] 
                             for f in families for d in days)

In [32]:
# Set objective function
mdl.minimize(preference_penalty + accounting_penalty)

mdl.print_information()

mdl.parameters.mip.tolerances.mipgap = 0.00
mdl.parameters.timelimit = 50000 
mdl.parameters.threads = 20

mdl.solve(log_output=True)
mdl.report()

preds = []
for (f, d) in sorted(assign_family_to_day_vars):
    if assign_family_to_day_vars[(f, d)].solution_value == 1:
        #print("Family %03d attends on day %03d" % (f, d))
        preds.append(d)

cost, _, _, _ = cost_function(preds)
cost

Model: santa
 - number of variables: 3584224
   - binary=3584224, integer=0, continuous=0
 - number of constraints: 653547
   - linear=653547
 - parameters:
     parameters.timelimit = 1000.00000000000000
 - problem type is: MILP
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 20
CPXPARAM_RandomSeed                              201903125
CPXPARAM_TimeLimit                               50000
CPXPARAM_MIP_Tolerances_MIPGap                   0
Tried aggregator 1 time.
MIP Presolve eliminated 653547 rows and 3584224 columns.
MIP Presolve modified 1699 coefficients.
All rows and columns eliminated.
Presolve time = 4.38 sec. (3715.76 ticks)

Root node processing (before b&c):
  Real time             =    5.23 sec. (4172.51 ticks)
Parallel b&c, 20 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
    

68888.04343194816

In [33]:
mdl.remove(sub_cst)

In [34]:
from docplex.mp.progress import SolutionRecorder

class MyProgressListener(SolutionRecorder):
    def __init__(self, model):
        SolutionRecorder.__init__(self)
        self.solutions = []
        self.current_objective = 999999;
    def notify_solution(self, s):
        SolutionRecorder.notify_solution(self, s)
        self.solutions.append(s)
        if self.current_progress_data.current_objective >= self.current_objective:
            return;
        self.current_objective = self.current_progress_data.current_objective;
        print ('Intermediate Solution')
        # solution = pd.DataFrame(data=[[f, d] for f in families for d in days if s.get_value(assign_day[f, d]) == 1],
        #                         columns=['family_id', 'assigned_day'])
        preds = []
        for (f, d) in sorted(assign_family_to_day_vars):
            if s.get_value(assign_family_to_day_vars[(f, d)]) == 1:
                preds.append(d)
        solution = pd.DataFrame(data=[f for f in families], columns = ['family_id'])
        solution['assigned_day'] = preds

        score, _, _, _ = cost_function(preds)
        print('Score: ' + str(score))
        solution.to_csv('../submissions/submission_' + str(score) + '.csv', index=False, line_terminator='\n', encoding='utf-8')
    def get_solutions(self):
        return self.solutions

In [35]:
listener = MyProgressListener(mdl)
mdl.add_progress_listener(listener)

In [36]:
mdl.add(preference_penalty + accounting_penalty <= 68888.05)
mdl.print_information()

Model: santa
 - number of variables: 3584224
   - binary=3584224, integer=0, continuous=0
 - number of constraints: 648548
   - linear=648548
 - parameters:
     parameters.threads = 20
     parameters.timelimit = 50000.00000000000000
     parameters.mip.tolerances.mipgap = 0.00000000000000
 - problem type is: MILP


In [37]:
mdl.parameters.mip.tolerances.mipgap = 0
mdl.parameters.timelimit = 1000000 
mdl.parameters.threads = 20
mdl.parameters.emphasis.mip = 2

mdl.solve(log_output=True)
mdl.report()

preds = []
for (f, d) in sorted(assign_family_to_day_vars):
    if assign_family_to_day_vars[(f, d)].solution_value == 1:
        #print("Family %03d attends on day %03d" % (f, d))
        preds.append(d)

cost, _, _, _ = cost_function(preds)
cost

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 20
CPXPARAM_RandomSeed                              201903125
CPXPARAM_Emphasis_MIP                            2
CPXPARAM_TimeLimit                               1000000
CPXPARAM_MIP_Tolerances_MIPGap                   0
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 68888.0434.
Tried aggregator 1 time.
Presolve has eliminated 603306 rows and 603306 columns...
MIP Presolve eliminated 603306 rows and 603306 columns.
MIP Presolve modified 111606 coefficients.
Reduced MIP has 45242 rows, 2980918 columns, and 10828020 nonzeros.
Reduced MIP has 2980918 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 20.95 sec. (14066.98 ticks)
Tried aggregator 1 time.
Presolve has eliminated 0 rows and 0 columns...
Detecting symmetries...
Reduced MIP has 45242 rows, 2980918 columns, and 10828020 

     36    26    68822.8968  1537    68888.0434    68746.7089   170735    0.21%
Elapsed time = 2556.85 sec. (1714358.25 ticks, tree = 0.46 MB, solutions = 1)
     37    25    68823.2951  1496    68888.0434    68746.7089   168388    0.21%
     39    13    68758.5524  1410    68888.0434    68746.7089   120943    0.21%
     40    28    68824.4525  1521    68888.0434    68746.7089   181823    0.21%
     41    31    68816.1515  1483    68888.0434    68747.5981   192048    0.20%
     43    17    68817.5563  1325    68888.0434    68747.5981   135090    0.20%
     44    29    68757.1700  1549    68888.0434    68747.5981   185356    0.20%
     45    35    68824.5518  1505    68888.0434    68747.5981   202926    0.20%
     47    34    68823.0179  1506    68888.0434    68747.5981   200412    0.20%
     48    33    68822.9006  1595    68888.0434    68747.5981   198672    0.20%
     49    19    68813.4763  1543    68888.0434    68747.5981   146914    0.20%
Elapsed time = 2611.92 sec. (1737254.81 ti

    446   355    68882.2056   725    68888.0434    68753.1947   774560    0.20%
    474   282    68827.0269  1475    68888.0434    68753.1947   680477    0.20%
    483   390    68824.7538  1606    68888.0434    68753.1947   830504    0.20%
    495   368    68859.1307   797    68888.0434    68753.1947   802775    0.20%
    511   392    68842.3622  1111    68888.0434    68753.1947   829507    0.20%
    531   394    68842.5195  1093    68888.0434    68753.1947   830859    0.20%
    552   361    68760.6852  1521    68888.0434    68753.1947   778029    0.20%
    569   378    68864.2122   825    68888.0434    68753.1947   810650    0.20%
Elapsed time = 3486.52 sec. (1995640.61 ticks, tree = 94.43 MB, solutions = 1)
    592   383    68880.7846   681    68888.0434    68753.1947   823841    0.20%
    617   485    68874.9189   836    68888.0434    68753.1947   935017    0.20%
    641   424        cutoff          68888.0434    68753.1947   875232    0.20%
    670   520        cutoff          6888

   2188  1456    68832.5291   972    68888.0434    68760.6853  2328143    0.18%
   2203  1550    68828.2977  1051    68888.0434    68760.6853  2454406    0.18%
   2216  1446    68800.4485  1163    68888.0434    68760.6853  2307839    0.18%
   2229  1446    68780.9224  1670    68888.0434    68760.6853  2312449    0.18%
Elapsed time = 4852.92 sec. (2359462.78 ticks, tree = 478.42 MB, solutions = 1)
   2247  1462    68847.0244   870    68888.0434    68762.5559  2333408    0.18%
   2262  1466    68854.2454   729    68888.0434    68762.5559  2334956    0.18%
   2282  1449    68782.0855  1624    68888.0434    68762.5559  2314796    0.18%
   2296  1478    68809.3711   933    68888.0434    68762.5559  2357786    0.18%
   2318  1472    68860.7114   712    68888.0434    68762.5559  2341578    0.18%
   2337  1498    68832.6788   899    68888.0434    68762.5559  2396995    0.18%
   2349  1667    68784.0037  1693    68888.0434    68762.5559  2558824    0.18%
   2364  1709    68818.0802  1012    688

Elapsed time = 6200.55 sec. (2721686.62 ticks, tree = 1076.82 MB, solutions = 1)
   4028  3040    68785.2033  1729    68888.0434    68766.0048  4036531    0.18%
   4047  2951    68792.7854  1615    68888.0434    68766.0048  3938464    0.18%
   4059  3060    68806.4204  1571    68888.0434    68766.0048  4051219    0.18%
   4081  2952    68793.1244  1643    68888.0434    68766.0048  3939053    0.18%
   4091  3111    68807.5796  1395    68888.0434    68766.0048  4111874    0.18%
   4110  2954    68794.4026  1546    68888.0434    68766.0048  3940735    0.18%
   4129  3183    68856.1400   932    68888.0434    68766.0916  4193593    0.18%
   4153  2875    68788.6917  1770    68888.0434    68766.0916  3861158    0.18%
   4177  3122    68787.3219  1679    68888.0434    68766.0916  4132875    0.18%
   4206  3234        cutoff          68888.0434    68766.0916  4263997    0.18%
Elapsed time = 6346.82 sec. (2761887.66 ticks, tree = 1280.57 MB, solutions = 1)
   4234  3233    68883.0015  1614    6

   4783     7    68828.8548  1319    68888.0434    68826.9625  5263892    0.09%
   4785     7        cutoff          68888.0434    68827.3134  5273836    0.09%
   4788    11        cutoff          68888.0434    68827.3143  5288471    0.09%
   4789     9    68846.6051   988    68888.0434    68827.3143  5270764    0.09%
   4791    12    68868.6917  1037    68888.0434    68827.3143  5299073    0.09%
Elapsed time = 9072.47 sec. (5224868.59 ticks, tree = 0.29 MB, solutions = 1)
   4794     7    68829.6068  1433    68888.0434    68827.3143  5264094    0.09%
   4795    13    68862.0516   993    68888.0434    68827.3143  5290467    0.09%
   4799    12    68829.9064  1267    68888.0434    68828.8565  5283808    0.09%
   4800    14    68870.3164  1072    68888.0434    68828.8565  5304529    0.09%
   4803    14    68871.8987  1020    68888.0434    68828.8609  5307270    0.09%
   4804    18    68868.6862   857    68888.0434    68828.8609  5318137    0.09%
   4810    15    68861.4341   985    68888

  11160  1371    68887.8869   660    68888.0434    68838.6044  6593350    0.07%
Elapsed time = 10107.68 sec. (5592521.40 ticks, tree = 269.71 MB, solutions = 1)
  11291  1559    68886.4834   873    68888.0434    68838.6044  6728104    0.07%
  11432  1570        cutoff          68888.0434    68838.6044  6745289    0.07%
  11596  1640    68887.0373   739    68888.0434    68838.6044  6820503    0.07%
  11757  1663    68887.3011   820    68888.0434    68838.6044  6841177    0.07%
  11933  1646        cutoff          68888.0434    68838.6044  6799494    0.07%
  12084  1623    68887.0622   658    68888.0434    68838.6044  6779348    0.07%
  12256  1711        cutoff          68888.0434    68838.6044  6874166    0.07%
  12430  1845    68887.5087   876    68888.0434    68838.6044  7016208    0.07%
  12580  1802        cutoff          68888.0434    68838.6044  6954228    0.07%
  12736  1842        cutoff          68888.0434    68838.6044  7095375    0.07%
Elapsed time = 10197.30 sec. (5630893.7

68888.04343194816

In [38]:
10615.48 / 3600


2.9487444444444444